### Process Parameters

In [12]:
def get_combinations(widths, heights):    
   
    sizes = []
    
    for width in widths:
        for height in heights:
            
            size = (width, height)
            reversed_size = (height, width)
                           
            if reversed_size in sizes:
                continue
                
            sizes.append(size)    
   
    sorted_sizes = tuple(sorted(sizes))    

    return sorted_sizes

In [13]:
def create_parameters(qml_device_types=None,
                      
                      image_width=None,
                      image_height=None,
                      
                      min_filter_width=None,
                      max_filter_width=None,
                      min_filter_height=None,
                      max_filter_height=None,
                      min_filters_count=1,
                      max_filters_count=1,

                      min_stride_width=1,
                      max_stride_width=1,
                      min_stride_height=1,
                      max_stride_height=1,

                      min_random_layers_count=1,
                      max_random_layers_count=1,

                      unique_filter_repetitions=False):
    
    if max_filter_width is None:        
        max_filter_width = min_filter_width

    if max_filter_height is None:        
        max_filter_height = min_filter_height
        
        
    # Calculate Secondary Parameters

    image_size = (image_width, image_height)

    max_filter_width = min(max_filter_width + 1, image_width + 1)
    max_filter_height = min(max_filter_height + 1, image_height + 1)

    filter_widths = list(range(min_filter_width, max_filter_width))
    filter_heights = list(range(min_filter_height, max_filter_height))

    filter_sizes = get_combinations(filter_widths, filter_heights)

    filter_counts = range(min_filters_count, max_filters_count + 1)

    filter_combinations = list(product(filter_sizes, filter_counts))

    random_layer_counts = list(range(min_random_layers_count, 
                                     max_random_layers_count + 1))

    stride_widths = range(min_stride_width, max_stride_width + 1)
    stride_heights = range(min_stride_height, max_stride_height + 1)

    strides = get_combinations(stride_widths, stride_heights)
    
    
    # Prepare Output
    
    parameters = {'qml_device_types': qml_device_types,
                  'image_size': image_size,
                  'filter_combinations': filter_combinations,
                  'strides': strides,
                  'random_layer_counts': random_layer_counts,
                  'unique_filter_repetitions': unique_filter_repetitions}
    
    return parameters

### Image Processing

In [4]:
def load_one_MNIST_image():

    # Load MNIST dataset

    mnist_dataset = keras.datasets.mnist
    train_data, test_data = mnist_dataset.load_data()

    train_images, train_labels = train_data
    test_images, test_labels = test_data

    # Normalize pixels

    train_images = train_images / 255
    test_images = test_images / 255

    # Add extra dimension for convolution channels

    train_images = np.array(train_images[..., tf.newaxis], requires_grad=False)
    test_images = np.array(test_images[..., tf.newaxis], requires_grad=False)

    one_image = train_images[0]
    
    return one_image

In [4]:
def parse_image(image_size, filter_size, stride_size, filters_count):
    
    image_width, image_height = image_size    
    filter_width, filter_height = filter_size
    stride_width, stride_height = stride_size
        
    filter_surface = filter_width * filter_height
    
    # Repetition counts
    
    if stride_width >= filter_width:
        
        horizontal_filter_repetitions = image_width // stride_width        
        horizontal_filter_repetitions += (image_width % stride_width) >= filter_width  

    else:        
       
        horizontal_filter_repetitions = (image_width - filter_width + 1) // stride_width  

        
    if stride_height >= filter_height:
        
        vertical_filter_repetitions = image_height // stride_height        
        vertical_filter_repetitions += (image_height % stride_height) >= filter_height
        
    else:
 
        vertical_filter_repetitions = (image_height - filter_height + 1) // stride_height
    
    
    filter_repetitions = horizontal_filter_repetitions * vertical_filter_repetitions
    
    filter_applications = filter_repetitions * filters_count

    complexity = (2 ** filter_surface
                  * filters_count
                  * filter_repetitions) 

    result = {"filter_width": filter_width, 
              "filter_height": filter_height,
              "filter_surface": filter_surface,
              "filters_count": filters_count,

              "stride_width": stride_width,
              "stride_height": stride_height,                      
              
              "horizontal_filter_repetitions": horizontal_filter_repetitions,
              "vertical_filter_repetitions": vertical_filter_repetitions,
              "filter_repetitions": filter_repetitions,              
              "filter_applications": filter_applications,   
              
              "complexity": complexity}
    
    return result

In [4]:
def get_pixel_values(image, experiment):
     
    filter_width = experiment['filter_width']
    filter_height = experiment['filter_height']  
    filters_count = experiment['filters_count']

    stride_width = experiment['stride_width']
    stride_height = experiment['stride_height']
    
    feature_width = horizontal_filter_repetitions = experiment['horizontal_filter_repetitions']    
    feature_height = vertical_filter_repetitions = experiment['vertical_filter_repetitions']
    
    channels_count = qubits_count = filter_surface = experiment['filter_surface']
   
    # Fetch pixels
    
    pixel_values_dict = dict()
    
    for feature_x in range(horizontal_filter_repetitions):
        
        image_corner_x = feature_x * stride_width
        
        for feature_y in range(vertical_filter_repetitions):
            
            image_corner_y = feature_y * stride_height
                
            image_fragment = image[image_corner_y : image_corner_y + filter_height,
                                   image_corner_x : image_corner_x + filter_width] 
            
            pixel_values_dict[(feature_x, feature_y)] = image_fragment.flatten()            
    
    pixel_values_array = jnp.array(list(pixel_values_dict.values()))
    
    return pixel_values_array

### Quantum

In [ ]:
def filter_circuit(pixel_values, random_layer_parameters):
        
    qubits = list(range(len(pixel_values)))

    for qubit, pixel_value in enumerate(pixel_values):     

        theta = jnp.pi * pixel_value        
        qml.RY(theta, wires=qubit)        
    
    RandomLayers(random_layer_parameters, wires=qubits)

    measurements = [qml.expval(qml.PauliZ(qubit)) for qubit in qubits]
    
    return measurements

### Parallelization

In [5]:
# Vectorizing Maps

def quanvolve_vmap(pixel_values_array, random_layer_parameters, filter_qnode):
   
    vmapped_circuit = jax.vmap(filter_qnode, in_axes=(0, None))
    
    vmapped_output = vmapped_circuit(pixel_values_array,
                                     random_layer_parameters)
       
    return vmapped_output


def quanvolve_no_vmap(pixel_values_array, random_layer_parameters, filter_qnode):        
  
    output = []
    
    for pixel_values in pixel_values_array:
        
        result = filter_qnode(pixel_values,
                              random_layer_parameters)
        
        output.append(result)
        
    output_array = jnp.array(output)
    
    return output_array

In [6]:
# Parallel Maps

def quanvolve_pmap(quanvolve, pixel_values_array, 
                   random_layer_parameters_array, filter_qnode):

    pmapped_quanvolve = jax.pmap(quanvolve,
                                 axis_name='filters',
                                 static_broadcasted_argnums=[2],
                                 in_axes=(None, 0, None))    

    filter_outputs = pmapped_quanvolve(pixel_values_array, 
                                       random_layer_parameters_array, 
                                       filter_qnode)        
    return filter_outputs


def quanvolve_no_pmap(quanvolve, pixel_values_array,
                      random_layer_parameters_array, filter_qnode):

    filter_outputs_list = []

    for random_layer_parameters in random_layer_parameters_array:

        filter_outputs_list.append(quanvolve(pixel_values_array,
                                             random_layer_parameters,
                                             filter_qnode))

    filter_outputs = np.array(filter_outputs_list)

    return filter_outputs

### Experiments

In [3]:
def prepare_experiments(parameters):
    
    JAX_COMPATIBLE_DEVICES = ["default.qubit.jax", "braket.aws.qubit"]
    
    # Parse Parameters
    
    qml_device_types = parameters['qml_device_types']
    strides = parameters['strides']    
    image_size = parameters['image_size']     
    filter_combinations = parameters['filter_combinations']
    random_layer_counts = parameters['random_layer_counts']
    unique_filter_repetitions = parameters['unique_filter_repetitions']       
    
    
    # Crete Experiments
    
    experiment_list = []

    for filter_size, filters_count in filter_combinations:

        for stride_size in strides:

            for random_layers_count in random_layer_counts:

                experiment = parse_image(image_size, filter_size, stride_size, filters_count)
                
                if unique_filter_repetitions:
                
                    skip_experiment = False

                    for existing_experiment in experiment_list:

                        if existing_experiment['filter_repetitions'] == experiment['filter_repetitions']:

                            skip_experiment = True
                            break

                    if skip_experiment:
                        continue

                experiment['random_layers_count'] = random_layers_count

                experiment_list.append(experiment)    

    sorted_experiments = sorted(experiment_list, 
                                key=lambda experiment: experiment['complexity'])
    
    # Update Device Info

    sorted_updated_experiments = []

    for device_type in qml_device_types:
         for experiment in sorted_experiments:

            if device_type in JAX_COMPATIBLE_DEVICES:
                interface = "jax"
                parallelize = True
                vectorize = True

            else:
                interface = "autograd"
                parallelize = False
                vectorize = False
                
            updated_experiment = {**experiment, 
                                  'device_type': device_type,
                                  'interface': interface,
                                  'parallelize': parallelize,
                                  'vectorize': vectorize}
                
            sorted_updated_experiments.append(updated_experiment)
            
            
    # Enumerated Dictionary

    experiments = dict(enumerate(sorted_updated_experiments))
    
    print(f"Experiments count: {len(experiments)}")
    
    return experiments

In [4]:
def run_experiment(experiment_id, experiment):   
   
    start_time = time.time()
    
    # Parse Experiment
    
    filter_width = experiment['filter_width']
    filter_height = experiment['filter_height']  
    filters_count = experiment['filters_count']

    stride_width = experiment['stride_width']
    stride_height = experiment['stride_height']
    
    feature_width = experiment['horizontal_filter_repetitions']    
    feature_height = experiment['vertical_filter_repetitions']    
   
    random_layers_count = experiment['random_layers_count']
    
    channels_count = qubits_count = filter_surface = experiment['filter_surface']
   
    device_type = experiment['device_type']
    interface = experiment['interface']
    
    parallelize_flag = experiment['parallelize']
    vectorize_flag = experiment['vectorize']
    
    
    # QML Device
    
    if device_type == 'lightning.gpu - multi-GPU':
        
        qml_device = qml.device('lightning.gpu', wires=qubits_count, batch_obs=True)
        
    elif device_type == 'braket.aws.qubit':

        qml_device = qml.device(
            name="braket.aws.qubit",
            device_arn="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
            wires=qubits_count,
            shots=1000,
            parallel=True)
        
    else:
        
        qml_device = qml.device(device_type, wires=qubits_count)
        
    
    # Filter QNode

    filter_qnode = qml.QNode(filter_circuit, qml_device, interface=interface)
    
    
    # Input parameters
    
    pixel_values_array = get_pixel_values(image, experiment)    
    
    random_layer_parameters_array =  np.random.uniform(high=2 * jnp.pi,
                                                       size=(filters_count,
                                                             random_layers_count,
                                                             channels_count))
    
    # Vectorize
    
    quanvolve = quanvolve_vmap if vectorize_flag else quanvolve_no_vmap

    # Parallelize
    
    quanvolve_over_filters = quanvolve_pmap if parallelize_flag else quanvolve_no_pmap
    

    filter_outputs = quanvolve_over_filters(quanvolve, pixel_values_array,
                                            random_layer_parameters_array, filter_qnode)
        
    feature_outputs = filter_outputs.reshape(filters_count, 
                                             feature_width, 
                                             -1, 
                                             channels_count)
#     # Printouts
    
#     print("pixel_values_array:", pixel_values_array)
#     print("random_layer_parameters_array:", random_layer_parameters_array)
#     print("filter_outputs:", filter_outputs)
#     print("feature_outputs:", feature_outputs)

#     print("pixel_values_array.shape:", pixel_values_array.shape)
#     print("random_layer_parameters_array.shape:", random_layer_parameters_array.shape)     
#     print("filter_outputs.shape:", filter_outputs.shape)  
#     print("feature_outputs.shape:", feature_outputs.shape)
    
      
    # Time
    
    execution_time = time.time() - start_time
    
    return execution_time

In [14]:
def run_experiments(experiments):
    
    if TRACK_AWS_BRAKET_COST:        
        aws_braket_cost_tracker = Tracker().start()

    experiments_count = len(experiments)

    for experiment_id, experiment in experiments.items():

        if 'execution_time' not in experiment:

            interface = experiment['interface']
            vectorize = experiment['vectorize']
            parallelize = experiment['parallelize']
            device_type = experiment['device_type']
            filters_count = experiment['filters_count']
            filter_surface = experiment['filter_surface']
            filter_repetitions = experiment['filter_repetitions']
            filter_applications = experiment['filter_applications']
            random_layers_count = experiment['random_layers_count']
            
            
            # Printouts

            print(f"Experiment: {experiment_id + 1}/{experiments_count}")
            # display(experiment)
            print(f"Device: {device_type}")
            # print(f"Interface: {interface}")
            # print(f"Vectorize: {vectorize}")
            # print(f"Parallelize: {parallelize}")
            # print(f"Filters count: {filters_count}")
            print(f"Qubits per filter: {filter_surface}")
            print(f"Filter repetitions: {filter_repetitions}")
            print(f"Filter applications: {filter_applications}")
            # print(f"Random layers count: {random_layers_count}")
            
            
            # Run

            execution_time = run_experiment(experiment_id, experiment)

            experiments[experiment_id]['execution_time'] = execution_time

            print(f"Execution time: {execution_time:.2f} seconds\n")
            
    if TRACK_AWS_BRAKET_COST:
        
        aws_braket_simulator_cost = aws_braket_cost_tracker.simulator_tasks_cost()        
        aws_braket_tasks_statistics = aws_braket_cost_tracker.quantum_tasks_statistics()
        
        print(f"AWS Braket Tasks Statistics:")
               
        for arn, result in aws_braket_cost_tracker.quantum_tasks_statistics().items():
    
            print('ARN:', arn)
            print('Tasks:', result['tasks'])
            print('Execution duration:', result['execution_duration'])
            print('Billed duration:   ', result['billed_execution_duration'])
            
        print(f"AWS Braket Simulator Cost: {aws_braket_simulator_cost:.04f} USD")

### Visualization

In [ ]:
def aggregate_execution_times(experiments):

    execution_times = defaultdict(lambda: defaultdict(list))

    for experiment_id, experiment in experiments.items():

        if 'execution_time' not in experiment:
            continue

        device_type = experiment['device_type']
        qubit_count = experiment['filter_surface']
        execution_time = experiment['execution_time']
        filter_repetitions = experiment['filter_repetitions']
        filter_applications = experiment['filter_applications']

        execution_times[device_type]['qubit_count'].append(qubit_count)
        execution_times[device_type]['execution_time'].append(execution_time)
        execution_times[device_type]['filter_repetitions'].append(filter_repetitions)
        execution_times[device_type]['filter_applications'].append(filter_applications)

    return execution_times

In [15]:
def plot_time_per_qubit_counts(execution_times):

    figure, (linear, logarithmic) = plt.subplots(1, 2, figsize=(12, 4))

    for device_type in execution_times:

        qubit_count = execution_times[device_type]['qubit_count']
        execution_time = execution_times[device_type]['execution_time']

        linear.plot(qubit_count[1:], 
                    execution_time[1:], 
                    '.-', label=device_type)

        logarithmic.plot(qubit_count[1:], 
                         execution_time[1:], 
                         '.-', label=device_type)

    linear.set_title("Linear")
    linear.set_ylabel("Execution time (s)")
    linear.set_xlabel("Qubits count")
    linear.legend()

    logarithmic.set_title("Logarithmic")
    logarithmic.set_xlabel("Qubits count")
    logarithmic.set_yscale("log")

    plt.show()

In [ ]:
def plot_time_per_filter_applications(execution_times):

    figure, (linear, logarithmic) = plt.subplots(1, 2, figsize=(12, 4))

    for device_type in execution_times:

        filter_applications = execution_times[device_type]['filter_applications']
        execution_time = execution_times[device_type]['execution_time']

        # print("filter_applications:", filter_applications)
        # print("execution_time:", execution_time)

        linear.plot(filter_applications[2:], 
                    execution_time[2:], 
                    '.-', label=device_type)

        logarithmic.plot(filter_applications[2:], 
                         execution_time[2:], 
                         '.-', label=device_type)

    linear.set_title("Linear")
    linear.set_ylabel("Execution time (s)")
    linear.set_xlabel("Filter applications")
    # linear.set_yscale("log")
    linear.legend()

    logarithmic.set_title("Logarithmic")
    logarithmic.set_xlabel("Filter applications")
    logarithmic.set_xscale("log")
    logarithmic.set_yscale("log")

    plt.show()

In [ ]:
def draw_filter_circuit(filter_circuit, wires_count):

    device = qml.device("default.qubit", wires=wires_count)

    filter_qnode = qml.QNode(filter_circuit, device)

    pixel_values = np.random.random(size=(wires_count,))
    random_layer_parameters = np.random.random(size=(wires_count, 1))

    qml.draw_mpl(filter_qnode, style='solarized_dark')(pixel_values, random_layer_parameters);

In [16]:
def plot_image(image):
    
    plt.figure(figsize=(2, 2))
    plt.imshow(image)
    plt.gca().set_axis_off()